<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Reinforcement Learning for Finance

**Chapter 02 &mdash; Deep Q-Learning**

&copy; Dr. Yves J. Hilpisch

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

### Please use the "Python 3.10, Tensorflow 2.10" kernel.

## CartPole

### The Game Environment

In [ ]:
!git clone https://github.com/tpq-classes/rl_4_finance.git
import sys
sys.path.append('rl_4_finance')


In [ ]:
import gymnasium as gym

In [ ]:
env = gym.make('CartPole-v1')

In [ ]:
env.action_space

In [ ]:
env.action_space.n  # <1>

In [ ]:
[env.action_space.sample() for _ in range(10)]  # <1>

In [ ]:
env.observation_space

In [ ]:
env.observation_space.shape  # <2>

In [ ]:
env.reset(seed=100)  # <1>

In [ ]:
env.step(0)  # <2>

In [ ]:
env.step(1)  # <2>

In [ ]:
class RandomAgent:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
    def play(self, episodes=1):
        self.trewards = list()
        for e in range(episodes):
            self.env.reset()
            for step in range(1, 100):
                a = self.env.action_space.sample()
                state, reward, done, trunc, info = self.env.step(a)
                if done:
                    self.trewards.append(step)
                    break

In [ ]:
ra = RandomAgent()

In [ ]:
ra.play(15)

In [ ]:
ra.trewards

In [ ]:
round(sum(ra.trewards) / len(ra.trewards), 2)  # <1>

In [ ]:
import os
import random
import warnings
import numpy as np
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from collections import deque
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0005)  # <2>

In [ ]:
tf.random.set_seed(100)

In [ ]:
# deque?

In [ ]:
class DQLAgent:
    def __init__(self):
        self.epsilon = 1.0  # <1>
        self.epsilon_decay = 0.9975  # <2>
        self.epsilon_min = 0.1  # <3>
        self.memory = list()  # <4>
        # self.memory = deque(maxlen=2000)
        self.batch_size = 32  # <5>
        self.gamma = 0.9  # <6>
        self.trewards = deque(maxlen=2000)  # <7>
        self.max_treward = 0  # <8>
        self._create_model()  # <9>
        self.env = gym.make('CartPole-v1')  # <10>
    def _create_model(self):
        self.model = Sequential()
        self.model.add(Dense(24, activation='relu', input_dim=4))
        self.model.add(Dense(24, activation='relu'))
        self.model.add(Dense(2, activation='linear'))
        self.model.compile(loss='mse', optimizer=opt)

In [ ]:
# Old verson, really slow
#class DQLAgent(DQLAgent):
#    def act(self, state):
#        if random.random() < self.epsilon:
#            return self.env.action_space.sample()  # <1>
#        return np.argmax(self.model.predict(state)[0])  # <2>
#    def replay(self):
#        batch = random.sample(self.memory, self.batch_size)  # <3>
#        for state, action, next_state, reward, done in batch:
#            if not done:
#                reward += self.gamma * np.amax(
#                    self.model.predict(next_state)[0])  # <4>
#            target = self.model.predict(state)  # <5>
#            target[0, action] = reward  # <6>
#            self.model.fit(state, target, epochs=2, verbose=False)  # <7>
#        if self.epsilon > self.epsilon_min:
#            self.epsilon *= self.epsilon_decay  # <8>

In [ ]:
# New Version, much faster
class DQLAgent(DQLAgent):
    def act(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()  # <1>
        q = self.model(tf.convert_to_tensor(state, dtype=tf.float32), training=False)
        return int(tf.argmax(q[0]).numpy())


    def replay(self):
        batch = random.sample(self.memory, self.batch_size)

        states      = np.vstack([b[0] for b in batch]).astype(np.float32)
        actions     = np.array([b[1] for b in batch], dtype=np.int32)
        next_states = np.vstack([b[2] for b in batch]).astype(np.float32)
        rewards     = np.array([b[3] for b in batch], dtype=np.float32)
        dones       = np.array([b[4] for b in batch], dtype=np.bool_)

        # Q(s, :)
        q_states = self.model(states, training=False).numpy()          # (B, A)

        # max_a' Q(s', a')
        q_next = self.model(next_states, training=False).numpy()       # (B, A)
        max_q_next = np.max(q_next, axis=1)                            # (B,)

        targets = q_states.copy()
        targets[np.arange(self.batch_size), actions] = rewards + self.gamma * max_q_next * (~dones)

         # One update only (FAST)
        self.model.train_on_batch(states, targets)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
class DQLAgent(DQLAgent):
    def learn(self, episodes):
        for e in range(1, episodes + 1):
            state, _ = self.env.reset()  # <1>
            state = np.reshape(state, [1, 4])  # <2>
            for f in range(1, 5000):
                action = self.act(state)  # <3>
                next_state, reward, done, trunc, _ = self.env.step(action)  # <4>
                next_state = np.reshape(next_state, [1, 4])  # <2>
                self.memory.append(
                    [state, action, next_state, reward, done])  # <4>
                state = next_state  # <5>
                if done or trunc:
                    self.trewards.append(f)  # <6>
                    self.max_treward = max(self.max_treward, f)  # <7>
                    templ = f'episode={e:4d} | treward={f:4d}'
                    templ += f' | max={self.max_treward:4d}'
                    print(templ, end='\r')
                    break
            if len(self.memory) > self.batch_size:
                self.replay()  # <8>
        print()

In [ ]:
# Old Version, really slow
#class DQLAgent(DQLAgent):
#    def test(self, episodes):
#        for e in range(1, episodes + 1):
#            state, _ = self.env.reset()
#            state = np.reshape(state, [1, 4])
#            for f in range(1, 5001):
#                action = np.argmax(self.model.predict(state)[0])  # <1>
#                state, reward, done, trunc, _ = self.env.step(action)
#                state = np.reshape(state, [1, 4])
#                if done or trunc:
#                    print(f, end=' ')
#                    break

In [ ]:
class DQLAgent(DQLAgent):
    def test(self, episodes):
        for e in range(1, episodes + 1):
            state, _ = self.env.reset()
            state = np.reshape(state, [1, 4])
            for f in range(1, 5001):
                action = np.argmax(self.model(tf.convert_to_tensor(state, dtype=tf.float32), training=False).numpy()[0])  # <1>
                state, reward, done, trunc, _ = self.env.step(action)
                state = np.reshape(state, [1, 4])
                if done or trunc:
                    print(f, end=' ')
                    break

In [ ]:
agent = DQLAgent()

In [ ]:
%time agent.learn(1000)

In [ ]:
agent.epsilon

In [ ]:
agent.test(15)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>